<a href="https://colab.research.google.com/github/LiamCarter2000/LiamCarter2000/blob/main/3%2B3_Simulations_1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
from typing_extensions import final

In [2]:
df = pd.read_csv("Decision Grid.csv", usecols=[1, 2, 3, 4, 5, 6, 7, 8])

df.columns = ['Treated', 'DLT', 'Pending', 'State', 'Orig_33_Action', 'IQ_33_Action', 'Orig_6_Action', 'IQ_6_Action']

decision_dict_IQ_33 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['IQ_33_Action'].to_dict()
decision_dict_33 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['Orig_33_Action'].to_dict()
decision_dict_IQ_6 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['IQ_6_Action'].to_dict()
decision_dict_6 = df.set_index(['Treated', 'DLT', 'Pending', 'State'])['Orig_6_Action'].to_dict()

In [3]:
# visualization of the processed data
print(df.head(15))

    Treated  DLT  Pending   State Orig_33_Action IQ_33_Action Orig_6_Action  \
0         0    0        0    open           Stay         Stay          Stay   
1         0    0        0  closed           Stay         Stay          Stay   
2         1    0        0    open           Stay         Stay          Stay   
3         1    0        0  closed           Stay         Stay          Stay   
4         1    1        0    open           Stay         Stay          Stay   
5         1    1        0  closed           Stay         Stay          Stay   
6         1    0        1    open           Stay         Stay          Stay   
7         1    0        1  closed           Stay         Stay          Stay   
8         2    0        0    open           Stay         Stay          Stay   
9         2    0        0  closed           Stay         Stay          Stay   
10        2    1        0    open           Stay         Stay          Stay   
11        2    1        0  closed           Stay    

In [4]:
print(f"Decision for Row index 29 3+3 (3,0,1,open): {decision_dict_33.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 IQ 3+3 (3,0,1,open): {decision_dict_IQ_33.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 rolling 6 (3,0,1,open): {decision_dict_IQ_6.get((3, 0, 1, 'open'))}")
print(f"Decision for Row index 29 IQ rolling 6 (3,0,1,open): {decision_dict_6.get((3, 0, 1, 'open'))}")


Decision for Row index 29 3+3 (3,0,1,open): Suspend
Decision for Row index 29 IQ 3+3 (3,0,1,open): Stay
Decision for Row index 29 rolling 6 (3,0,1,open): Stay
Decision for Row index 29 IQ rolling 6 (3,0,1,open): Stay


Data is processed into four python dictionaries: one for each of the methodologies for which the simulation will run (original 3+3, IQ 3+3, original rolling 6, IQ rolling 6). The values in each of the columns of these methodologies are as follows.

 Defines what dose level the next patient should be treated at, based on the current number of patients being treated, the number of reported DLTs, and the number of results pending at the current dose level.

Possible Dispositions:

1. Up          = start testing patients at the next higher dose level
2. Down     = start testing patients at the next lower dose level
3. Stay       = keep testing patients at the current dose level
4. Suspend = wait for additional testing to finish at current dose
                      level, before starting a test on another patient.
5. MTD       = current dose is the Maximum Tolerated Dose (MTD)


The value of having this as a python dictionary is that all values can be looked up in place for a much faster run time. So if we want to know what to do for IQ 3+3 when we have Treated = 3, DLT = 0, Pending = 1, and State = open, we can just query the corresponding dictionary with "decision_dict_IQ_33.get((3, 0, 1, 'open'))" to get "Stay"

All that is left is to simulate the actions of the dispositions while also considering possible dropout of people that develop DLT.




In [5]:
Starting_Dose_Level = 2
Lowest_Possible_Dose_Level = 1
Highest_Possible_Dose_Level = 5

#(maximum duration interested patients will wait for a slot)
Waitlist_Time = 0
#(duration of DLT evaluation period)
Course_Length = 28
#(percent who will fail the screening process)
Screen_Failure_Prob = 0.30
#(percent inevaluable.  Actual inevaluable rate will be lower due to competing DLT risk)
Inevaluable_Prob = 0.20

#DLT Probability Function (defines the probability of a patient having a DLT event, based on dose)
def get_dlt_probability(dose_level):
    """Using values from the paper"""
    # Formula: 100 * (0.5 + atan(0.2 * pi * (curr_dose_level - 8.5)) / pi)
    val = 0.5 + (math.atan(0.2 * math.pi * (dose_level - 8.5)) / math.pi)
    return max(0.0, min(1.0, val))

#Patient Inter-Arrival Time (days between arrivals)
def sample_interarrival():
    return math.floor(np.random.exponential(10))  # Mean of 10 days

#Screening Duration Distribution (length of screening process)
def sample_screening_duration():
    # beta(0, 28, 1, 1) is a uniform distribution between 0 and 28
    return math.floor(np.random.uniform(0, 28))

#Time Until DLT Event (distribution of times before the patient will have the DLT event)
def sample_dlt_timing():
    # beta(0, CourseLength, 1.5, 1) skewed toward the end of the cycle
    return math.floor(np.random.beta(1.5, 1) * Course_Length)

#Time Until Inevaluable (distribution of times before the patient will become inevaluable)
def sample_inevaluable_timing():
    # beta(0,CourseLength,1,1) is a Uniform(0, 28) distribution
    return math.floor(np.random.beta(1, 1) * Course_Length)



Now to make a class for the average patient

In [6]:
class Patient:
  def __init__(self, arrival_time, dose, ID, course_length = Course_Length):
    self.ID = ID
    self.arrival_time = arrival_time
    self.dose = dose #int 1-5 inclusive
    self.course_length = course_length #length of study
    self.started_treatment = False
    self.treatment_start_time = None

    self.screening_outcome, self.days_to_screening_outcome = self.determine_screening_outcome()

    self.started_screening = False
    self.screening_start_time = None
    self.screening_end_time = None

    self.treatment_outcome, self.days_to_treatment_outcome = self.determine_patient_outcome()

    self.started_treatment = False
    self.treatment_start_time = None
    self.resolution_time = None


  def determine_screening_outcome(self):
    if np.random.random() < Screen_Failure_Prob:
      return "fail", 0
    else:
      return "pass", sample_screening_duration()

  def start_screening(self, current_clock_time):
    self.started_screening = True
    self.screening_start_time = current_clock_time
    self.screening_end_time = self.screening_start_time + self.days_to_screening_outcome


  # returns "dlt", "ineval", or "pass", as well as a duration til it happens
  def determine_patient_outcome(self):
    dlt_prob = get_dlt_probability(self.dose)
    ineval_prob = Inevaluable_Prob

    time_to_ineval = -1
    time_to_dlt = -1

    if np.random.random() < ineval_prob:
      time_to_ineval = sample_inevaluable_timing()

    if np.random.random() < dlt_prob:
      time_to_dlt = sample_dlt_timing()

    # if these both happen, we choose the one that will happen first in the timeline
    if time_to_ineval > -1 and time_to_dlt > -1:
      if time_to_ineval < time_to_dlt:
        return "ineval", time_to_ineval
      else:
        return "dlt", time_to_dlt
    elif time_to_ineval > -1:
      return "ineval", time_to_ineval
    elif time_to_dlt > -1:
      return "dlt", time_to_dlt
    else:
      return "pass", self.course_length

  # current_clock_time will be an int representation of days since trial
  # resolution_time is the day they pass, get dlt, or become ineval
  def start_treatment(self, current_clock_time):
    self.started_treatment = True
    self.treatment_start_time = current_clock_time
    self.resolution_time = self.treatment_start_time + self.days_to_treatment_outcome


In [7]:
def archive(treated_list, pending_list):
  treated_list.extend(pending_list)
  pending_list.clear()


In [8]:

def simulation_normal(trial_type):
  if trial_type == 1:
    decision_dict = decision_dict_33
    slot_limit = 3
  elif trial_type == 3:
    decision_dict = decision_dict_6
    slot_limit = 6

  # used to properly implement the slot limitatons of a study (cohort management)
  batch_counter = 0

  # debug error var
  end_on_error = False

  trial_status = True
  # options are Stay, Up, Down, Suspend, MTD
  current_trial_event = "Stay"
  dose_level = Starting_Dose_Level
  max_dose_level = Highest_Possible_Dose_Level
  min_dose_level = Lowest_Possible_Dose_Level
  final_dose_level = None

  # this will be how many patients are not taken because of lack of a queue
  overlooked_patients = 0
  # starts on day 1 of trial
  current_clock_time = 1
  # always need one at the start
  patient_needed = True
  days_before_next_patient = sample_interarrival()

  patientID = 1

  screening_list = []
  pending_list = []
  # (list of patients all patients: treated, DLT, and pending)
  treated_list = []
  patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "open"}
  total_inevals = 0

  while trial_status and current_clock_time < 2000:
    """
    check for new patients and add them to the screening list if they are desired
    check the screening list patients and add them to the patient list if they are ready
    check the patients in the pending list to see if they have passed or otherwise and act accordingly
    check for trial status change
      do stuff if changed
    """

    # checking for new patients, add them to screening
    # without queue, turned away if not needed that day
    # Patient cannot start screening unless they are needed, lest we waste their time
    if days_before_next_patient <= 0 and patient_needed and batch_counter < slot_limit:
      days_before_next_patient = sample_interarrival()
      new_patient = Patient(current_clock_time, dose_level, patientID)
      patientID += 1
      new_patient.start_screening(current_clock_time)
      screening_list.append(new_patient)
      # uf they are screening, they take up a spot in the study
      batch_counter += 1
    elif days_before_next_patient == 0:
      days_before_next_patient = sample_interarrival()
      overlooked_patients += 1
    else:
      days_before_next_patient -= 1


    # checking screening list (patient queued if not needed, no need to waste a screened patient, dose level subject to change if Up or Down)
    #!!! for non-IQ we do need to iterate each patient in screening to see if they are all done, for IQ you might not want all on same day, but you do here
    #!!! for patient in screening_list[:]: """this does break some logic though so leaving out 2 patients added in one day"""
    if screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list[0]
      if patient.screening_outcome == "fail":
        screening_list.pop(0)
        batch_counter -= 1
      elif patient.screening_outcome == "pass" and current_trial_event != "Suspend":
        screening_list.pop(0)
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
        patient_states["pending"] += 1
        patient_states["treated"] += 1


    # checking pending list
    for patient in pending_list[:]:
      # will never trigger twice cause clock time changes. check will happen every time though, could optomize to event checking
      # no mention of "pass" because num treated alredy reflects that patient
      if patient.resolution_time == current_clock_time:
        pending_list.remove(patient)
        patient_states["pending"] -= 1
        # inevals not archived, just noted as gone
        # decrement because they do leave the study
        if patient.treatment_outcome == "ineval":
          patient_states["treated"] -= 1
          total_inevals += 1
          batch_counter -= 1
          continue
        elif patient.treatment_outcome == "dlt":
          patient_states["DLT"] += 1
        treated_list.append(patient)

    # options are Stay, Up, Down, Suspend, MTD
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

    if current_trial_event == "Stay":
      patient_needed = True
      # cohort expansion logic for when extra cohort needed to determine MTD
      if trial_type == 1 and batch_counter == 3 and patient_states['pending'] == 0 and screening_list == []:
        batch_counter = 0
    elif current_trial_event == "Suspend":
      patient_needed = False
    elif current_trial_event == "Up":
      archive(treated_list, pending_list)

      max_completed_dose_level = dose_level
      dose_level += 1
      patient_needed = True
      batch_counter = 0

      new_state = "closed" if dose_level == max_dose_level else "open"
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": new_state})

    elif current_trial_event == "Down":
      archive(treated_list, pending_list)

      dose_level -= 1
      patient_needed = True
      batch_counter = 0

      # this dose level is now considered toxic
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"})

    elif current_trial_event == "MTD":
      archive(treated_list, pending_list)

      final_dose_level = dose_level
      trial_status = False

    # haven't triggered this block in a long time, yay
    elif current_trial_event == "Error":
      archive(treated_list, pending_list)
      final_dose_level = dose_level

      if dose_level == min_dose_level:
        result_summary = "Terminated: Too Toxic at Lowest Level"
      else:
        result_summary = "Terminated: Error/Inconclusive"

      print("Error in chart reached")
      print(f"Reason: {result_summary}")
      end_on_error = True
      trial_status = False

    current_clock_time += 1


  # returns list of all treated/dlt patients, days_taken, final_dose_level, total_inevals
  return treated_list, current_clock_time, final_dose_level, total_inevals


In [9]:
def simulation_IQ(trial_type):
  if trial_type == 2:
    decision_dict = decision_dict_IQ_33
  elif trial_type == 4:
    decision_dict = decision_dict_IQ_6

  #slot limit not needed for IQ
  # debug error var
  end_on_error = False

  trial_status = True
  # options are Stay, Up, Down, Suspend, MTD
  current_trial_event = "Stay"
  dose_level = Starting_Dose_Level
  max_dose_level = Highest_Possible_Dose_Level
  min_dose_level = Lowest_Possible_Dose_Level
  final_dose_level = None

  # this will be how many patients are not taken because of lack of a queue
  overlooked_patients = 0
  # starts on day 1 of trial
  current_clock_time = 1
  # always need one at the start
  patient_needed = True
  #initializing first arrival day
  days_before_next_patient = sample_interarrival()

  patientID = 1

  screening_list = []
  pending_list = []
  # (list of patients all patients: treated, DLT, and pending)
  treated_list = []
  patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "open"}
  total_inevals = 0

  while trial_status and current_clock_time < 2000:
    """
    check for new patients and add them to the screening list if they are desired
    check the screening list patients and add them to the patient list if they are ready
    check the patients in the pending list to see if they have passed or otherwise and act accordingly
    check for trial status change
      do stuff if changed
    """

    # checking for new patients, add them to screening
    # without queue, turned away if not needed that day
    # Patient cannot start screening unless they are needed, lest we waste their time
    if days_before_next_patient <= 0 and patient_needed:
      days_before_next_patient = sample_interarrival()
      new_patient = Patient(current_clock_time, dose_level, patientID)
      patientID += 1
      new_patient.start_screening(current_clock_time)
      screening_list.append(new_patient)
    elif days_before_next_patient == 0:
      days_before_next_patient = sample_interarrival()
      overlooked_patients += 1
    else:
      days_before_next_patient -= 1



    # checking screening list
    # Use 'while' to allow multiple ready patients to start on the same day
    while screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list[0]
      if patient.screening_outcome == "fail":
        screening_list.pop(0)
        # Move to the next person in the list for the same day
        continue
      current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

      if patient.screening_outcome == "pass" and current_trial_event != "Suspend":
        screening_list.pop(0)
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
        patient_states["pending"] += 1
        patient_states["treated"] += 1
      else:
        break


    # checking pending list
    for patient in pending_list[:]:
      # will never trigger twice cause clock time changes. check will happen every time though, could optomize to event checking
      # no mention of "pass" because num treated alredy reflects that patient
      if patient.resolution_time == current_clock_time:
        pending_list.remove(patient)
        patient_states["pending"] -= 1
        # inevals not archived, just noted as gone
        # decrement because they do leave the study
        if patient.treatment_outcome == "ineval":
          patient_states["treated"] -= 1
          total_inevals += 1
          continue
        elif patient.treatment_outcome == "dlt":
          patient_states["DLT"] += 1
        treated_list.append(patient)

    # options are Stay, Up, Down, Suspend, MTD
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

    if current_trial_event == "Stay":
      patient_needed = True
    elif current_trial_event == "Suspend":
      patient_needed = False
    elif current_trial_event == "Up":
      archive(treated_list, pending_list)

      max_completed_dose_level = dose_level
      dose_level += 1
      patient_needed = True

      new_state = "closed" if dose_level == max_dose_level else "open"
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": new_state})

    elif current_trial_event == "Down":
      archive(treated_list, pending_list)

      dose_level -= 1
      patient_needed = True

      # this dose level is now considered toxic
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"})

    elif current_trial_event == "MTD":
      archive(treated_list, pending_list)

      final_dose_level = dose_level
      trial_status = False

    # haven't triggered this block in a long time, yay
    elif current_trial_event == "Error":
      archive(treated_list, pending_list)
      final_dose_level = dose_level

      if dose_level == min_dose_level:
        result_summary = "Terminated: Too Toxic at Lowest Level"
      else:
        result_summary = "Terminated: Error/Inconclusive"

      print("Error in chart reached")
      print(f"Reason: {result_summary}")
      end_on_error = True
      trial_status = False

    current_clock_time += 1


  # returns list of all treated/dlt patients, days_taken, final_dose_level, total_inevals
  return treated_list, current_clock_time, final_dose_level, total_inevals


In [10]:
def simulation(trial_type):
  if trial_type == 1 or trial_type == 3:
    return simulation_normal(trial_type)
  elif trial_type == 2 or trial_type == 4:
    return simulation_IQ(trial_type)
  else:
    print("Invalid trial type: Please choose a value 1-4")
    return None, None, None, None

In [11]:
# example of metrics being pulled from treated patients
treated_list, days_taken, final_dose_level, total_inevals = simulation(1)
print(f"Final Dose Level: {final_dose_level}")
print(f"Total Time Taken: {days_taken} days or ~{days_taken // 30} months")
print(f"Total Inevals: {total_inevals}")
print(f"Treated List: {len(treated_list)}")
print("---First treated patient below---")
print(vars(treated_list[0]))

Final Dose Level: 5
Total Time Taken: 656 days or ~21 months
Total Inevals: 6
Treated List: 18
---First treated patient below---
{'ID': 1, 'arrival_time': 3, 'dose': 2, 'course_length': 28, 'started_treatment': True, 'treatment_start_time': 18, 'screening_outcome': 'pass', 'days_to_screening_outcome': 15, 'started_screening': True, 'screening_start_time': 3, 'screening_end_time': 18, 'treatment_outcome': 'pass', 'days_to_treatment_outcome': 28, 'resolution_time': 46}


In [12]:
# function takes simulation_type (1 = 3+3, 2 = IQ 3+3, 3 = rolling 6, 4 = IQ rolling 6), num_repititions
# returns metrics_df = dataframe of a dict formatted like this: [{"days_taken": 0, "final_dose": 0, "num_inevals": 0, "num_treated": 0, "num_DLT_above_MTD": 0},...]

def get_metrics(simulation_type, num_repititions):
  metrics = []

  for i in range(num_repititions):
    if simulation_type == 1 or simulation_type == 3:
      treated_list, days_taken, final_dose_level, total_inevals = simulation(simulation_type)
    elif simulation_type == 2 or simulation_type == 4:
      treated_list, days_taken, final_dose_level, total_inevals = simulation_IQ(simulation_type)
    num_DLT_above_MTD = 0

    for patient in treated_list:
      if patient.treatment_outcome == "dlt" and patient.dose > final_dose_level:
        num_DLT_above_MTD += 1

    trial_data = {
        "days_taken": days_taken,
        "final_dose": final_dose_level,
        "num_inevals": total_inevals,
        "num_treated": len(treated_list),
        "num_DLT_above_MTD": num_DLT_above_MTD
    }

    metrics.append(trial_data)

  metrics_df = pd.DataFrame(metrics)

  return metrics_df

In [13]:
def bucket_dose(val):
  if val is None or val <= 0:
    return "NA"
  return int(val)

# Reindex to ensure all levels are shown and format strings
def format_pct(val):
  if val == 0:
    return "0.0%"
  if 0 < val < 1.0:
    return "<1.0%"
  return f"{val:.1f}%"

# for non python coders, 'apply' applies a function to a group of values. it implicitly sends the parameter.
# Creates the MTD percentage breakdown chart from a single column/list of doses.
def get_mtd_breakdown(doses, max_level=5):
  bucketed = doses.apply(bucket_dose)

  # Calculate percentages (total length of the series is the denominator)
  probs = (bucketed.value_counts() / len(doses) * 100)

  display_order = ["NA"] + list(range(1, max_level + 1))

  chart = probs.reindex(display_order, fill_value=0.0).apply(format_pct)

  return chart.to_frame(name="% MTD At level")

def get_report(simulation_type, num_repititions):
  metrics_df = get_metrics(simulation_type, num_repititions)

  doses = metrics_df['final_dose'].copy()
  doses_breakdown = get_mtd_breakdown(doses)

  metrics_df['months_taken'] = metrics_df['days_taken'] / 30.4375

  summary = metrics_df.agg({
      'num_treated': ['mean', 'median', 'min', 'max'],
      'months_taken': ['mean', 'median', 'min', 'max'],
      'num_DLT_above_MTD': ['mean']
  })

  summary['num_treated'] = summary['num_treated'].round(1)
  summary['months_taken'] = summary['months_taken'].round(1)
  summary['num_DLT_above_MTD'] = summary['num_DLT_above_MTD'].round(2)

  summary.loc['range'] = summary.apply(lambda col: f"{col['min']} - {col['max']}")

  final_report = summary.drop(['min', 'max'])

  if simulation_type == 1:
    print("metrics for 3+3")
  elif simulation_type == 2:
    print("metrics for IQ 3+3")
  elif simulation_type == 3:
    print("metrics for Rolling 6")
  elif simulation_type == 4:
    print("metrics for IQ Rolling 6")

  print(final_report)
  print("-----")
  print(doses_breakdown)
  print("--------------------\n")

Dr Frankel

# This is the code block that runs the metrics that compare directly to the paper's supplimental

the first input is the model type, and the second is the number of simulations

1: 3+3

2: IQ 3+3

3: R6

4: IQR6


In [14]:
# TODO: add additional metric as input/output, such as Ineval rate and other things to better compare to other parts of the study
Inevaluable_Prob = 0.20
Starting_Dose_Level = 2
Highest_Possible_Dose_Level = 5
Lowest_Possible_Dose_Level = 1

get_report(1, 8000)

#get_report(2, 800)

# get_report(3, 8000)

# get_report(4, 8000)


metrics for 3+3
       num_treated months_taken num_DLT_above_MTD
mean          17.0         20.4              0.99
median        17.0         19.9               NaN
range   7.0 - 38.0   5.2 - 47.7         nan - nan
-----
           % MTD At level
final_dose               
NA                  <1.0%
1                    7.1%
2                    9.0%
3                   12.4%
4                   13.6%
5                   57.5%
--------------------



In [15]:
# debug_chart = None
# for i in range(10):
#   print(f"Trial {i}")
#   chart, end_on_error = simulation(2)
#   if end_on_error:
#     debug_chart = pd.DataFrame(chart)
#     break
#   print("-----------\n-----------")

# print(debug_chart)


In [16]:
def debug_log(day, event, action, states):
  print(f"Day: {day:<4} | Event: {event:<10} | {action:<65} | States: {states}")


from typing_extensions import final
def simulation_with_debug(trial_type):
  if trial_type == 1:
    decision_dict = decision_dict_33
    slot_limit = 3
    print("Beginning 3+3 Simulation")
  elif trial_type == 2:
    decision_dict = decision_dict_IQ_33
    slot_limit = 8
    print("Beginning IQ 3+3 Simulation")
  elif trial_type == 3:
    decision_dict = decision_dict_6
    slot_limit = 6
    print("Beginning Rolling 6 Simulation")
  elif trial_type == 4:
    decision_dict = decision_dict_IQ_6
    slot_limit = 8
    print("Beginning IQ Rolling 6 Simulation")


   # used to properly implement the slot limitatons of a study (cohort management)
  batch_counter = 0

  # debug error var
  end_on_error = False

  trial_status = True
  # options are Stay, Up, Down, Suspend, MTD
  current_trial_event = "Stay"
  dose_level = Starting_Dose_Level
  max_dose_level = Highest_Possible_Dose_Level
  min_dose_level = Lowest_Possible_Dose_Level
  final_dose_level = None

  # this will be how many patients are not taken because of lack of a queue
  overlooked_patients = 0
  # starts on day 1 of trial
  current_clock_time = 1
  # always need one at the start
  patient_needed = True
  days_before_next_patient = sample_interarrival()

  patientID = 1

  screening_list = []
  pending_list = []
  # (list of patients all patients: treated, DLT, and pending)
  treated_list = []
  patient_states = {"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "open"}
  total_inevals = 0

  # log (for debug: not needed for many simulations in a row, wasted compute)
  state_log = []

  while trial_status and current_clock_time < 2000:
    """
    check for new patients and add them to the screening list if they are desired
    check the screening list patients and add them to the patient list if they are ready
    check the patients in the pending list to see if they have passed or otherwise and act accordingly
    check for trial status change
      do stuff if changed
    """

    # checking for new patients, add them to screening
    # without queue, turned away if not needed that day
    if days_before_next_patient <= 0 and patient_needed and batch_counter < slot_limit:
      days_before_next_patient = sample_interarrival()
      new_patient = Patient(current_clock_time, dose_level, patientID)
      patientID += 1
      new_patient.start_screening(current_clock_time)
      screening_list.append(new_patient)
      # if they are screening, they take up a spot in the study
      batch_counter += 1
      debug_log(current_clock_time, current_trial_event, f"Adding new patient {new_patient.ID} to screening.", patient_states)
    elif days_before_next_patient == 0:
      days_before_next_patient = sample_interarrival()
      overlooked_patients += 1
    else:
      days_before_next_patient -= 1

    # checking screening list (patient queued if not needed, no need to waste a screened patient, dose level subject to change if Up or Down)
    # no need for a loop if we just want to add 1 at most on a given day
    if screening_list and screening_list[0].screening_end_time <= current_clock_time:
      patient = screening_list[0]
      if patient.screening_outcome == "fail":
        screening_list.pop(0)
        batch_counter -= 1
        debug_log(current_clock_time, current_trial_event, f"Patient {patient.ID} failed screening.", patient_states)
      elif patient.screening_outcome == "pass" and current_trial_event != "Suspend":
        screening_list.pop(0)
        if patient.dose != dose_level:
          patient.dose = dose_level
        patient.start_treatment(current_clock_time)
        pending_list.append(patient)
        patient_states["pending"] += 1
        patient_states["treated"] += 1
        debug_log(current_clock_time, current_trial_event, f"Adding patient {patient.ID} to pending from screening, at dose level {patient.dose}.", patient_states)


    # checking pending list
    for patient in pending_list[:]:
      # will never trigger twice cause clock time changes. check will happen every time though, could optomize to event checking
      # no mention of "pass" because num treated alredy reflects that patient
      if patient.resolution_time == current_clock_time:
        pending_list.remove(patient)
        patient_states["pending"] -= 1
        if patient.treatment_outcome == "pass":
          debug_log(current_clock_time, current_trial_event, f"Patient {patient.ID} passes treatment.", patient_states)
        # inevals not archived, just noted as gone
        # decrement because they do leave the study
        if patient.treatment_outcome == "ineval":
          patient_states["treated"] -= 1
          total_inevals += 1
          batch_counter -= 1
          debug_log(current_clock_time, current_trial_event, f"Removing patient {patient.ID} for ineval.", patient_states)
          continue
        elif patient.treatment_outcome == "dlt":
          patient_states["DLT"] += 1
          debug_log(current_clock_time, current_trial_event, f"Patient {patient.ID} develops DLT.", patient_states)
        treated_list.append(patient)

    # options are Stay, Up, Down, Suspend, MTD
    current_trial_event = decision_dict.get((patient_states['treated'], patient_states['DLT'], patient_states['pending'], patient_states['AboveLevelState']), "Error")

    if current_trial_event == "Stay":
      patient_needed = True
      # if extra cohort needed for 33 mtd
      if trial_type == 1 and batch_counter == 3 and patient_states['pending'] == 0 and screening_list == []:
        batch_counter = 0
        debug_log(current_clock_time, current_trial_event, f"Expanding cohort to 6 at dose level: {dose_level}", patient_states)
    elif current_trial_event == "Suspend":
      patient_needed = False
    elif current_trial_event == "Up":
      ids = [p.ID for p in pending_list]
      debug_log(current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, dose goes up.", patient_states)

      archive(treated_list, pending_list)

      max_completed_dose_level = dose_level
      dose_level += 1
      patient_needed = True
      batch_counter = 0

      new_state = "closed" if dose_level == max_dose_level else "open"
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": new_state})

      # print(f"dose escalated to : {dose_level}")
    elif current_trial_event == "Down":
      ids = [p.ID for p in pending_list]
      debug_log(current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, dose goes down.", patient_states)

      archive(treated_list, pending_list)

      dose_level -= 1
      patient_needed = True
      batch_counter = 0

      # this dose level is now considered toxic
      patient_states.update({"treated": 0, "DLT": 0, "pending": 0, "AboveLevelState": "closed"})

      # print(f"Dose de-escalated to : {dose_level}")
    elif current_trial_event == "MTD":
      ids = [p.ID for p in pending_list]
      debug_log(current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, dose is MTD.", patient_states)

      archive(treated_list, pending_list)

      final_dose_level = dose_level
      trial_status = False

      # print(f"MTD Reached on day {current_clock_time}, with dose level {final_dose_level}")
    elif current_trial_event == "Error":
      ids = [p.ID for p in pending_list]
      debug_log(current_clock_time, current_trial_event, f"Moving patients {ids} to treated from pending, error.", patient_states)

      archive(treated_list, pending_list)
      final_dose_level = dose_level

      if dose_level == min_dose_level:
        result_summary = "Terminated: Too Toxic at Lowest Level"
      else:
        result_summary = "Terminated: Error/Inconclusive"

      print("Error in chart reached")
      print(f"Reason: {result_summary}")
      end_on_error = True
      trial_status = False


    snapshot = patient_states.copy()
    snapshot['day'] = current_clock_time
    snapshot['dose'] = dose_level
    snapshot['action'] = current_trial_event
    state_log.append(snapshot)

    current_clock_time += 1


  # print(f"Final Dose Level: {final_dose_level}")
  # print(f"Final Patient States: {patient_states}")
  # print(f"Total Time Taken: {current_clock_time} days or ~{current_clock_time // 30} months")
  # print(f"Overlooked patients: {overlooked_patients}")

  # return state_log if you want a break down of the states day by day
  # returns list of all treated/dlt patients, days_taken, final_dose_level, total_inevals
  return treated_list, current_clock_time, final_dose_level, total_inevals, state_log


Dr Frankel

## Click Play Below to generate a simulation.
# The int parameter 1-4 controls which simulation you are running

1: 3+3

2: IQ 3+3

3: R6

4: IQR6

The box below this one shows a day by day state table for the simulation you just ran. Click the calculator looking symbol to look through it

In [17]:
treated_list, days_taken, final_dose_level, total_inevals, state_log = simulation_with_debug(4)


Beginning IQ Rolling 6 Simulation
Day: 30   | Event: Stay       | Adding new patient 1 to screening.                                | States: {'treated': 0, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'open'}
Day: 30   | Event: Stay       | Patient 1 failed screening.                                       | States: {'treated': 0, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'open'}
Day: 39   | Event: Stay       | Adding new patient 2 to screening.                                | States: {'treated': 0, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'open'}
Day: 39   | Event: Stay       | Patient 2 failed screening.                                       | States: {'treated': 0, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'open'}
Day: 46   | Event: Stay       | Adding new patient 3 to screening.                                | States: {'treated': 0, 'DLT': 0, 'pending': 0, 'AboveLevelState': 'open'}
Day: 51   | Event: Stay       | Adding new patient 4 to screening.                              

In [18]:
log_df = pd.DataFrame(state_log)
log_df

,treated,DLT,pending,AboveLevelState,day,dose,action
0,0,0,0,open,1,2,Stay
1,0,0,0,open,2,2,Stay
2,0,0,0,open,3,2,Stay
3,0,0,0,open,4,2,Stay
4,0,0,0,open,5,2,Stay
...,...,...,...,...,...,...,...
452,6,1,1,closed,453,5,Stay
453,6,1,1,closed,454,5,Stay
454,6,1,1,closed,455,5,Stay
455,6,1,1,closed,456,5,Stay
